# Library

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
import pyexasol
import psycopg2

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
wd = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='vsyse')

In [4]:
# exasol db
# wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
# we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
# wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# Load
- 등록&제원 -> result 4등급(연료 : 경유, 휘발유, LPG)

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [5]:
# 8.6s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")

In [6]:
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1927232 entries, 0 to 1927231
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   VIN             object 
 1   BSPL_STDG_CD    int64  
 2   VHCL_ERSR_YN    object 
 3   MANP_MNG_NO     object 
 4   YRIDNW          int64  
 5   VHCTY_CD        object 
 6   PURPS_CD2       object 
 7   FRST_REG_YMD    float64
 8   VHCL_FBCTN_YMD  float64
 9   VHCL_MNG_NO     object 
dtypes: float64(2), int64(2), object(6)
memory usage: 147.0+ MB


In [7]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호',
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자',
    'VHCL_MNG_NO':'차량관리번호'
}

In [8]:
carr = car.rename(columns=car_ch_col)

In [9]:
carr.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호'],
      dtype='object')

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [10]:
# 3.8s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2 FROM STD_CEG_CAR_SRC_MIG;")

In [11]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
}

In [12]:
srcr = src.rename(columns=src_ch_col)
srcr.columns

Index(['제원관리번호', '연료', '차종유형'], dtype='object')

## 저감장치 부착이력(STD_DLM_TB_ERP_ATT_HIS)

In [13]:
# 3.0s
att = wd.export_to_pandas("SELECT VIN, RDCDVC_SE_CD FROM STD_DLM_TB_ERP_ATT_HIS;")
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013842 entries, 0 to 1013841
Data columns (total 2 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   VIN           1013842 non-null  object
 1   RDCDVC_SE_CD  1013842 non-null  object
dtypes: object(2)
memory usage: 15.5+ MB


In [14]:
att_ch_col = {
    'VIN':'차대번호', 
    'RDCDVC_SE_CD':'저감장치구분', 
}

In [15]:
attr = att.rename(columns=att_ch_col)
attr.columns

Index(['차대번호', '저감장치구분'], dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [16]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [17]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [18]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [19]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## 등록이력(CEG_CAR_HISTORY_MIG)

In [45]:
# 1.8s
edb_id = 'vsysd'
edb_database = 'edb'
edb_port = 5444
edb_url = '172.29.135.50'
edb_pwd = 'vsyswynn'
conn = psycopg2.connect(dbname=edb_database, user=edb_id, password=edb_pwd, host=edb_url, port=edb_port)
cur = conn.cursor()
sql = 'select VHCL_ERSR_YN, CHNG_DE, VHMNO from vsysd.ceg_car_history_mig'
cur.execute(sql)
his = pd.DataFrame(cur.fetchall())
his.columns = [desc[0].upper() for desc in cur.description]
cur.close()
conn.close()

In [46]:
# # 1m 8.2s
# his = ws.export_to_pandas("SELECT VHCL_ERSR_YN, CHNG_DE, VHMNO FROM CEG_CAR_HISTORY_MIG;")
# his.info()

In [47]:
his_ch_col = {
    'VHCL_ERSR_YN':'차량말소YN', 
    'CHNG_DE':'변경일자',
    'VHMNO':'차량관리번호'
}

In [48]:
hisr = his.rename(columns=his_ch_col)
hisr.columns

Index(['차량말소YN', '변경일자', '차량관리번호'], dtype='object')

## RH에서 제공한 법정동코드

In [49]:
rh = we.export_to_pandas("SELECT DONG_CODE, CTPRVN_NM, SIGNGU_NM FROM TB_MAPDATA;")
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   DONG_CODE  278 non-null    int64 
 1   CTPRVN_NM  278 non-null    object
 2   SIGNGU_NM  260 non-null    object
dtypes: int64(1), object(2)
memory usage: 6.6+ KB


In [72]:
rh = rh.rename(columns={'DONG_CODE':'법정동코드_rh', 'CTPRVN_NM':'시도', 'SIGNGU_NM':'시군구'})
rh.head()

,법정동코드_rh,시도,시군구
0,0,전국,NaN
1,1100000000,서울특별시,NaN
2,1111000000,서울특별시,종로구
3,1114000000,서울특별시,중구
4,1117000000,서울특별시,용산구


In [73]:
rh.shape

(278, 3)

## 4등급 result(for DPF유무)

In [52]:
rs = we.export_to_pandas("SELECT 차대번호, DPF유무_수정 FROM GRD4_RESULT;")
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


# 전처리

## 중복 차대번호 제거

In [53]:
carr.shape, len(carr['차대번호'].unique())

((1927232, 10), 1927178)

In [54]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1927178, 10)

## 차종 코드 변환

In [55]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['승용', '화물', '승합', '특수'], dtype=object)

## 용도 코드 변환

In [56]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [57]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1401060, 10)

## 연료 코드 변환

In [58]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['휘발유', '경유', '기타연료', 'LPG(액화석유가스)', nan, 'CNG(압축천연가스)',
       '휘발유 하이브리드', '알코올', '전기', 'CNG 하이브리드', '태양열', '경유 하이브리드', '등유',
       'LPG 하이브리드', 'LNG(액화천연가스)', '수소'], dtype=object)

## 차종유형 코드 변환

In [59]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '중형', '소형', '경형', nan], dtype=object)

## 등록&제원 병합

In [60]:
# 19.4s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1401060, 12)

## 저감장치구분 코드 변환

In [61]:
# 4m 14.3s
attr['저감장치구분'].unique()

array(['A1005', 'A1001', 'A1003', 'A1002', 'A1007', 'A1004', 'A1006'],
      dtype=object)

In [62]:
# 27.5s
rdcdvc_dict = {
    'A1001':'1종', 
    'A1002':'2종', 
    'A1003':'3종', 
    'A1004':'1종+SCR', 
    'A1005':'엔진개조', 
    'A1006':'엔진교체',
    'A1007':'삼원촉매',
}
attr['저감장치구분'] = attr['저감장치구분'].replace(rdcdvc_dict)
attr['저감장치구분'].unique()

array(['엔진개조', '1종', '3종', '2종', '삼원촉매', '1종+SCR', '엔진교체'], dtype=object)

## 저감장치 부착 유무

In [63]:
attr.loc[(attr['저감장치구분'] == '1종') | (attr['저감장치구분'] == '1종+SCR'), 'DPF_YN'] = '유'

In [64]:
attr['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    533073
유      480769
Name: count, dtype: int64

## 등록&제원&저감이력 병합

In [65]:
# 1.7s
csa = cs.merge(attr[['차대번호', 'DPF_YN']], on='차대번호', how='left')
csa.shape

(1401060, 13)

## 지역정보 병합

In [66]:
df = csa.merge(coder, on='법정동코드', how='left')

### 매칭 안되는 지역 처리

In [67]:
# 경기도 양주시
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시']

### rh 법정동코드 참고하여 법정동코드 수정

In [68]:
df.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구'],
      dtype='object')

In [69]:
rdf = df.copy()

In [70]:
rdf['법정동코드'] = rdf['법정동코드'].astype('str')
rdf['법정동코드_mod'] = rdf['법정동코드'].str[:5] + '00000'
rdf['법정동코드_mod'] = pd.to_numeric(rdf['법정동코드_mod'])
rdf['법정동코드_mod'].head()

0    3120000000
1    4167000000
2    4374000000
3    4374000000
4    4374000000
Name: 법정동코드_mod, dtype: int64

In [74]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

{2772000000,
 2817000000,
 4119500000,
 4119700000,
 4119900000,
 4173000000,
 4371000000,
 5111000000,
 5113000000,
 5115000000,
 5117000000,
 5119000000,
 5121000000,
 5123000000,
 5172000000,
 5173000000,
 5175000000,
 5176000000,
 5177000000,
 5178000000,
 5179000000,
 5180000000,
 5181000000,
 5182000000,
 5183000000}

In [75]:
# 2817000000 인천광역시 남구 -> 인천광역시 미추홀구 2817700000
# 4119500000 경기도 부천시 원미구 -> 경기도 부천시 4119000000
# 4119700000 경기도 부천시 소사구 -> 경기도 부천시 4119000000
# 4119900000 경기도 부천시 오정구 -> 경기도 부천시 4119000000
# 4173000000 경기도 여주군 -> 경기도 여주시 4167000000
# 4371000000 충청북도 청원군 -> 충청북도 충주시 4311000000

In [76]:
rdf.loc[rdf['법정동코드_mod'] == 2817000000, '법정동코드_mod'] = 2817700000
rdf.loc[rdf['법정동코드_mod'] == 4119500000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119700000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119900000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4173000000, '법정동코드_mod'] = 4167000000
rdf.loc[rdf['법정동코드_mod'] == 4371000000, '법정동코드_mod'] = 4311000000

In [77]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

{2772000000,
 5111000000,
 5113000000,
 5115000000,
 5117000000,
 5119000000,
 5121000000,
 5123000000,
 5172000000,
 5173000000,
 5175000000,
 5176000000,
 5177000000,
 5178000000,
 5179000000,
 5180000000,
 5181000000,
 5182000000,
 5183000000}

In [78]:
rdf.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod'],
      dtype='object')

In [79]:
rdf['법정동코드_mod'].isnull().sum()

0

In [80]:
df = rdf.copy()
df.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod'],
      dtype='object')

## 4등급 result 파일 참고하여 DPF유무 수정

In [81]:
rdf = df.copy()

In [82]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

2100

In [83]:
rdf['DPF_YN'].value_counts(dropna=False)

DPF_YN
NaN    1401005
유           55
Name: count, dtype: int64

In [84]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879481
NaN     348422
유       261897
확인불가     12435
Name: count, dtype: int64

In [85]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [86]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879481
NaN     348400
유       261897
확인불가     12435
Name: count, dtype: int64

In [87]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [88]:
rdf1.loc[(rdf1['DPF_YN'] == '유') | (rdf1['DPF유무_수정'] == '유'), 'DPF_YN'] = '유'
rdf1.loc[(rdf1['DPF유무_수정'] == '무'), 'DPF_YN'] = '무'
rdf1.loc[(rdf1['DPF유무_수정'] == '확인불가'), 'DPF_YN'] = '확인불가'

In [89]:
rdf1['DPF_YN'].value_counts(dropna=False)

DPF_YN
무       813815
NaN     339024
유       236809
확인불가     11412
Name: count, dtype: int64

In [90]:
rdf1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod',
       'DPF유무_수정'],
      dtype='object')

In [91]:
df = rdf1.drop('DPF유무_수정', axis=1)

In [92]:
df.shape

(1401060, 16)

## 연료 확인

In [93]:
df['연료'].value_counts(dropna=False)

연료
경유             1062218
휘발유             314285
LPG(액화석유가스)      24472
NaN                 49
기타연료                34
CNG(압축천연가스)          1
알코올                  1
Name: count, dtype: int64

## 연료 컬럼 추가

In [94]:
df1 = df.copy()

In [95]:
df1.loc[df1['연료'] == '경유', 'fuel'] = '경유'
df1.loc[(df1['연료'] == '휘발유') | (df1['연료'] == 'LPG(액화석유가스)'), 'fuel'] = '휘발유_가스'

# 분석

## EG 분류

In [96]:
df1['fuel'].value_counts(dropna=False)

fuel
경유        1062218
휘발유_가스     338757
NaN            85
Name: count, dtype: int64

In [97]:
grade_list = []
for f, y, cy, e in tqdm(df1[['fuel', '제작일자', '차량연식', 'DPF_YN']].values):
    if (f == '휘발유_가스') and ( (19980101 <= y <= 20001231) or (1998 <= cy <= 2000) ):
        grade_list.append('A')
    elif (f == '휘발유_가스') and ( (y <= 19971231) or (cy <= 1997) ):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '유'):
        grade_list.append('A')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) )and (e == '유'):
        grade_list.append('B')
    elif (f == '경유') and ( (y >= 20080101) or (cy >= 2008) ) and (e == '무'):
        grade_list.append('C')
    elif (f == '경유') and ( (y <= 20071231) or (cy <= 2007) ) and (e == '무'):
        grade_list.append('D')
    else:
        grade_list.append('X')
len(grade_list)

100%|██████████| 1401060/1401060 [00:02<00:00, 619081.58it/s]


1401060

In [98]:
df1['Grade'] = grade_list

In [99]:
df1.columns

Index(['차대번호', '법정동코드', '차량말소YN', '제원관리번호', '차량연식', '차종', '용도', '최초등록일자',
       '제작일자', '차량관리번호', '연료', '차종유형', 'DPF_YN', '시도', '시군구', '법정동코드_mod',
       'fuel', 'Grade'],
      dtype='object')

In [100]:
df2 = df1[[
    '차대번호', 
    '제원관리번호',
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도', 
    '시군구', 
    'DPF_YN',
    'Grade',
    '법정동코드_mod',
    ]]
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드', '시도',
       '시군구', 'DPF_YN', 'Grade', '법정동코드_mod'],
      dtype='object')

In [101]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'20230711'

In [102]:
df2['테이블생성일자'] = today_date

/tmp/ipykernel_1316254/2409603865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['테이블생성일자'] = today_date


In [103]:
df2.columns

Index(['차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드', '시도',
       '시군구', 'DPF_YN', 'Grade', '법정동코드_mod', '테이블생성일자'],
      dtype='object')

In [104]:
df3 = df2[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도', 
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구', 
    'DPF_YN', 
    'Grade', 
    '법정동코드_mod',
    ]]
df3.columns

Index(['테이블생성일자', '차대번호', '제원관리번호', '차종', '용도', '차량연식', '차종유형', '연료', '법정동코드',
       '시도', '시군구', 'DPF_YN', 'Grade', '법정동코드_mod'],
      dtype='object')

In [105]:
df3['Grade'].value_counts(dropna=False)

Grade
C    780933
B    366615
A    195316
D     32882
X     25314
Name: count, dtype: int64

In [106]:
df3['법정동코드_mod'] = df3['법정동코드_mod'].astype('str')

In [107]:
ch_col_dict = {
                '기준연월':'LOAD_DT',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO',
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2',
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM',
                '시군구':'SGG_NM',
                'DPF_YN':'DPF_MNTNG_YN',
                'Grade':'GRD4_MLSFC', 
                '법정동코드_mod':'STDG_CD_MOD'
                }

In [108]:
STD_BD_GRD4_MLSFC_RSLT = df3.rename(columns=ch_col_dict)
STD_BD_GRD4_MLSFC_RSLT.columns

Index(['테이블생성일자', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2', 'YRIDNW',
       'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'DPF_MNTNG_YN',
       'GRD4_MLSFC', 'STDG_CD_MOD'],
      dtype='object')

In [109]:
STD_BD_GRD4_MLSFC_RSLT.tail()

,테이블생성일자,VIN,MANG_MNG_NO,VHCTY_CD,PURPS_CD2,YRIDNW,VHCTY_TY,FUEL_CD,STDG_CD,CTPV_NM,SGG_NM,DPF_MNTNG_YN,GRD4_MLSFC,STDG_CD_MOD
1401055,20230711,KMHJF31JPPU555457,10022700140000,승용,개인용,1993,소형,휘발유,1132010600,서울특별시,도봉구,NaN,B,1132000000
1401056,20230711,KMHVF31JPMU511183,99931107001641103,승용,개인용,1991,소형,휘발유,1117012400,서울특별시,용산구,NaN,B,1117000000
1401057,20230711,KNADA2423MS619775,99931107044681103,승용,개인용,1991,소형,휘발유,1117012400,서울특별시,용산구,NaN,B,1117000000
1401058,20230711,KMHVF21JPMU415752,99934103019601103,승용,개인용,1991,소형,휘발유,4122025000,경기도,평택시,NaN,B,4122000000
1401059,20230711,KNADA23K2NS120848,99934125002081103,승용,개인용,1992,소형,휘발유,4122025000,경기도,평택시,NaN,B,4122000000


## [출력] STD_BD_GRD4_MLSFC_RSLT

In [110]:
expdf = STD_BD_GRD4_MLSFC_RSLT
table_nm = 'STD_BD_GRD4_MLSFC_RSLT'.upper()

# 테이블 생성
sql = 'create or replace table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 7s
we.import_from_pandas(expdf, table_nm)

# 분석2

In [ ]:
carr.columns

In [ ]:
hisr.columns

## 등록 & 제원 정보 병합(말소 유지)

In [ ]:
carr.shape

In [ ]:
csersr = carr.merge(srcr[['제원관리번호', '연료']], on='제원관리번호', how='left')
csersr.shape

## 1\. 차량관리번호 기준 병합

In [ ]:
csersr.shape

In [ ]:
# 58.3s
ersr = csersr.merge(hisr, on='차량관리번호', how='left')
ersr.shape

## 2\. 차량말소YN 만 추출

In [ ]:
errm = ersr[(ersr['차량말소YN_x'] == 'Y') & (ersr['차량말소YN_y'] == 'Y')].reset_index(drop=True)
errm.shape

## 3\. 변경일자 최신으로 차대번호 중복 제거

In [ ]:
errm = errm.sort_values('변경일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)

## 4\. 변경일자 2019.01.01 이상만 추출

In [ ]:
errm = errm[errm['변경일자'] >= 20190101].reset_index(drop=True)

In [ ]:
errm.shape

In [ ]:
errm.columns

## 5\. 법정동 코드 정보 병합

In [ ]:
errc = errm.merge(coder, on='법정동코드', how='left')

## 시도, 연도별 차량 대수

In [ ]:
df['최초등록일자'].describe()

In [ ]:
dfm = df.copy()

In [ ]:
dfm['최초등록일자'] = dfm['최초등록일자'].astype('str')

In [ ]:
dfm['최초등록일자_년'] = dfm['최초등록일자'].str[:4]
dfm['최초등록일자_월'] = dfm['최초등록일자'].str[4:6]
dfm['최초등록일자_일'] = dfm['최초등록일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(dfm)

In [ ]:
# 시군구명 앞쪽 지역명만 남기기(dfm)
dfm['시군구_수정'] = dfm['시군구'].str.split(' ').str[0]

### 연료 지역별 등록차량대수

In [ ]:
num_car_by_local = dfm.groupby(['연료', '시도', '시군구_수정', '최초등록일자_년', '최초등록일자_월'], as_index=False)['차대번호'].count()
num_car_by_local = num_car_by_local.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도', '최초등록일자_월':'월'})
num_car_by_local

### 연료 지역별 차량대수

In [ ]:
num_car_by_local2 = dfm.loc[dfm['최초등록일자'] < '20220601'].groupby(['연료', '시도', '시군구_수정'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'차량대수'})
num_car_by_local2

In [ ]:
num_car_by_local2[['연도', '월']] = ['2022', '06']
num_car_by_local2

### 연료 지역별 말소 대수

In [ ]:
errc['변경일자'].head()

In [ ]:
errc['변경일자'] = errc['변경일자'].astype('str')

In [ ]:
errc['변경일자_년'] = errc['변경일자'].str[:4]
errc['변경일자_월'] = errc['변경일자'].str[4:6]
errc['변경일자_일'] = errc['변경일자'].str[6:8]

### 시군구명 앞쪽 지역명만 남기기(errc)

In [ ]:
# 시군구명 앞쪽 지역명만 남기기(errc)
errc['시군구_수정'] = errc['시군구'].str.split(' ').str[0]

In [ ]:
grp_erase = errc.loc[errc['변경일자_년'] == '2022'].groupby(['변경일자_년', '변경일자_월', '연료', '시도', '시군구_수정'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도', '변경일자_월':'월'})
grp_erase = grp_erase.sort_values(['시도', '시군구_수정'])
grp_erase

In [ ]:
grp_erase[grp_erase['연도'] == '2022'].shape

In [ ]:
num_car_by_local2['연도'].max()

In [ ]:
yr_list = []
mth_list = []
fuel_list = []
ctpv_list = []
sgg_list = []
yr = int(num_car_by_local2['연도'].max())
mth = int(num_car_by_local2['월'].max())
sl = num_car_by_local2.drop_duplicates(['시도', '시군구_수정']).reset_index(drop=True)
for ctpv, sgg in sl[['시도', '시군구_수정']].values:
    for fuel in sl['연료'].unique():
        for mth in range(mth - 4, mth + 1):
            mthm = f'{mth:0>2}'
            yr_list.append(str(yr))
            mth_list.append(mthm)
            fuel_list.append(fuel)
            ctpv_list.append(ctpv)
            sgg_list.append(sgg)
base = pd.DataFrame({'연도':yr_list, '월':mth_list, '연료':fuel_list, '시도':ctpv_list, '시군구_수정':sgg_list})

In [ ]:
base

In [ ]:
num_car_by_local2.head()

In [ ]:
base1 = base.merge(num_car_by_local2, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [ ]:
base2 = base1.merge(num_car_by_local, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [ ]:
base3 = base2.merge(grp_erase, on=['연도', '월', '연료', '시도', '시군구_수정'], how='left')

In [ ]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

In [ ]:
n = 5
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']
    # base3.loc[(i+1)*n - 3, '차량대수'] = base3.loc[(i+1)*n - 2, '차량대수'] + base3.loc[(i+1)*n - 2, '말소차량대수'] - base3.loc[(i+1)*n - 2, '등록차량대수']
    # base3.loc[(i+1)*n - 4, '차량대수'] = base3.loc[(i+1)*n - 3, '차량대수'] + base3.loc[(i+1)*n - 3, '말소차량대수'] - base3.loc[(i+1)*n - 3, '등록차량대수']
    # base3.loc[(i+1)*n - 5, '차량대수'] = base3.loc[(i+1)*n - 4, '차량대수'] + base3.loc[(i+1)*n - 4, '말소차량대수'] - base3.loc[(i+1)*n - 4, '등록차량대수']

In [ ]:
base3.head()

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

In [ ]:
base3['테이블생성일자'] = today_date

In [ ]:
base4 = base3[[
    '테이블생성일자', 
    '기준연월',
    '연도',
    '월', 
    '연료', 
    '시도', 
    '시군구_수정', 
    '차량대수',
]]

In [ ]:
chc_col = {
    '테이블생성일자':'LOAD_DT', 
    '기준연월':'CRTR_YM',
    '연도':'YR', 
    '월':'MM', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '시군구_수정':'SGG', 
    '차량대수':'VHCL_MKCNT', 
}

In [ ]:
STD_BD_GRD4_RGN_CURSTT = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT.columns

### [출력] STD_BD_GRD4_RGN_CURSTT

In [ ]:
# expdf = STD_BD_GRD4_RGN_CURSTT
# table_nm = 'STD_BD_GRD4_RGN_CURSTT'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# 분석3

In [ ]:
dfm.columns

In [ ]:
errc.columns

## 연도 시도 차종별 차량 대수

### 현재 차량 대수

In [ ]:
num_car_by_local1 = dfm.loc[dfm['최초등록일자'] < '20220601'].groupby(['연료', '시도', '차종'], dropna=False)['차대번호'].count().reset_index()
num_car_by_local1 = num_car_by_local1.rename(columns={'차대번호':'차량대수'})
num_car_by_local1

In [ ]:
num_car_by_local1['연도'] = '2022'
num_car_by_local1

### 등록 차량 대수

In [ ]:
num_car_by_local2 = dfm.groupby(['연료', '시도', '차종', '최초등록일자_년'], as_index=False)['차대번호'].count()
num_car_by_local2 = num_car_by_local2.rename(columns={'차대번호':'등록차량대수', '최초등록일자_년':'연도'})
num_car_by_local2

### 말소 차량 대수

In [ ]:
grp_erase = errc.groupby(['변경일자_년', '연료', '시도', '차종'], as_index=False)['차대번호'].count()
grp_erase = grp_erase.rename(columns={'차대번호':'말소차량대수', '변경일자_년':'연도'})
grp_erase = grp_erase.sort_values(['시도'])
grp_erase

In [ ]:
yr_list = []
fuel_list = []
ctpv_list = []
cd_list = []
yr = int(num_car_by_local2['연도'].max())
for ctpv in num_car_by_local1['시도'].unique():
    for fuel in sl['연료'].unique():
        for cd in ['승용', '승합', '화물', '특수']:
            for yrm in range(yr - 3, yr + 1):
                yr_list.append(str(yrm))
                fuel_list.append(fuel)
                ctpv_list.append(ctpv)
                cd_list.append(cd)
base = pd.DataFrame({'연도':yr_list, '연료':fuel_list, '시도':ctpv_list, '차종':cd_list})

In [ ]:
base

In [ ]:
base1 = base.merge(num_car_by_local1, on=['연도', '연료', '시도', '차종'], how='left')
base1

In [ ]:
base2 = base1.merge(num_car_by_local2, on=['연도', '연료', '시도', '차종'], how='left')
base2

In [ ]:
base3 = base2.merge(grp_erase, on=['연도', '연료', '시도', '차종'], how='left')
base3

In [ ]:
base3[['차량대수', '등록차량대수', '말소차량대수']] = base3[['차량대수', '등록차량대수', '말소차량대수']].fillna(0)
base3[['차량대수', '등록차량대수', '말소차량대수']].isnull().sum()

In [ ]:
n = 4
for i in range(base3.shape[0] // n):
    for j in range(2, n+1):
        base3.loc[(i+1)*n - j, '차량대수'] = base3.loc[(i+1)*n - (j-1), '차량대수'] + base3.loc[(i+1)*n - (j-1), '말소차량대수'] - base3.loc[(i+1)*n - (j-1), '등록차량대수']
    # base3.loc[(i+1)*n - 3, '차량대수'] = base3.loc[(i+1)*n - 2, '차량대수'] + base3.loc[(i+1)*n - 2, '말소차량대수'] - base3.loc[(i+1)*n - 2, '등록차량대수']
    # base3.loc[(i+1)*n - 4, '차량대수'] = base3.loc[(i+1)*n - 3, '차량대수'] + base3.loc[(i+1)*n - 3, '말소차량대수'] - base3.loc[(i+1)*n - 3, '등록차량대수']
    # base3.loc[(i+1)*n - 5, '차량대수'] = base3.loc[(i+1)*n - 4, '차량대수'] + base3.loc[(i+1)*n - 4, '말소차량대수'] - base3.loc[(i+1)*n - 4, '등록차량대수']

In [ ]:
today_date = datetime.today().strftime("%Y%m")
today_date

In [ ]:
base3['테이블생성일자'] = today_date

In [ ]:
base4 = base3[[
    '테이블생성일자', 
    '연도', 
    '연료', 
    '시도', 
    '차종', 
    '차량대수', 
]]

In [ ]:
chc_col = {
    '테이블생성일자':'LOAD_DT',
    '연도':'YR', 
    '연료':'FUEL_CD', 
    '시도':'CTPV', 
    '차량대수':'VHCL_MKCNT', 
}

In [ ]:
STD_BD_GRD4_RGN_CURSTT_MOD = base4.rename(columns=chc_col)
STD_BD_GRD4_RGN_CURSTT_MOD.columns

In [ ]:
STD_BD_GRD4_MLSFC_RSLT.head()

### [출력] STD_BD_GRD4_RGN_CURSTT_MOD

In [ ]:
# expdf = STD_BD_GRD4_RGN_CURSTT_MOD
# table_nm = 'STD_BD_GRD4_RGN_CURSTT_MOD'.upper()

# # 테이블 생성
# sql = 'create or replace table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# we.execute(sql)

# # 데이터 추가
# # 5s
# we.import_from_pandas(expdf, table_nm)

# 코드 마지막